In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import mean_squared_error

def rmse(y_te, y_p):
    return np.sqrt(mean_squared_error(y_te, y_p))

In [3]:
data = pd.read_pickle('data.pkl')
data = data[data.date_block_num > 11]

In [4]:
X = data[data.date_block_num < 34].drop(['item_cnt_month'], axis=1)
Y = data[data.date_block_num < 34]['item_cnt_month']
x = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [5]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.72, random_state=42)

lr = LinearRegression()
score = cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
print('RMSE %2f' %(-1 * score.mean()))

RMSE 0.825634
